# Assingment 2 - Tumor segmentation
### Course: Convolutional Neural Networks with Applications in Medical Image Analysis

Previously you have looked at the different available contrasts of the same anatomy, but the dataset also contains a manually segmented binary map of a tumor, if visible on the slice. For the second assignment your task is to create a tumor segmentation model that takes any number of image contrasts as an input, and outputs the tumor segmentations. For this task, you will implement and use the [DICE](https://en.wikipedia.org/wiki/S%C3%B8rensen%E2%80%93Dice_coefficient) score to evaluate (aim for a DICE score higher than $0.8$ on the validation set).

Your task is to look through the highly customizable code below, which contains all the main steps for image segmentation from using only one contrast as an input. By changing the arrays of the DataGenerator, multiple contrasts can be added as input, similar as in Assignment 1. The most important issues with the current code are noted in the comments for easier comprehension. Your tasks, to include in the Jupyter notebook you hand in, are:
- How you reached the required performances (DICE score above 0.8)
- Plot the training/validating losses and accuracies. Describe when to stop training, and why that is a good choice.
- Once you have reached the required loss on the validation data, and you have selected your final model, only then evaluate your model on the testing data as well.
- Describe the thought process behind building your model and choosing the model hyper-parameters.
- Describe what you think are the biggest issues with the provided network and training, and how you solved them.

Upload the updated notebook to Canvas by March $31^{th}$, 15:00.

Good luck and have fun!

In [ ]:
import os

import numpy as np
np.random.seed(2022)  # Set seed for reproducibility

import tensorflow as tf
tf.random.set_seed(2022)

from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
import tensorflow.keras as keras

import matplotlib.pyplot as plt

gpus = tf.config.experimental.list_physical_devices('GPU')
if len(gpus) > 0:
    tf.config.experimental.set_memory_growth(gpus[0], True)
    print(f"GPU(s) available (using '{gpus[0].name}'). Training will be lightning fast!")
else:
    print("No GPU(s) available. Training will be suuuuper slow!")

# NOTE: These are the packages you will need for the assignment.
# NOTE: You are encouraged to use the course virtual environment, which already has GPU support.

##### The cell below will define the data generator for the data you will be using. You should not change anything in the below code!

In [ ]:
class DataGenerator(keras.utils.Sequence):
    def __init__(self,
                 data_path,
                 arrays,
                 batch_size=32,
                 ):

        self.data_path = data_path
        self.arrays = arrays
        self.batch_size = batch_size

        if data_path is None:
            raise ValueError('The data path is not defined.')

        if not os.path.isdir(data_path):
            raise ValueError('The data path is incorrectly defined.')

        self.file_idx = 0
        self.file_list = [self.data_path + '/' + s for s in
                          os.listdir(self.data_path)]
        
        self.on_epoch_end()
        with np.load(self.file_list[0]) as npzfile:
            self.in_dims = []
            self.n_channels = 1
            for i in range(len(self.arrays)):
                im = npzfile[self.arrays[i]]
                self.in_dims.append((self.batch_size,
                                    *np.shape(im),
                                    self.n_channels))

    def __len__(self):
        """Get the number of batches per epoch."""
        return int(np.floor((len(self.file_list)) / self.batch_size))

    def __getitem__(self, index):
        """Generate one batch of data."""
        # Generate indexes of the batch
        indexes = self.indexes[index * self.batch_size:(index + 1) *
                               self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.file_list[k] for k in indexes]

        # Generate data
        a = self.__data_generation(list_IDs_temp)
        return a

    def on_epoch_end(self):
        """Update indexes after each epoch."""
        self.indexes = np.arange(len(self.file_list))
        np.random.shuffle(self.indexes)
    
    #@threadsafe_generator
    def __data_generation(self, temp_list):
        """Generate data containing batch_size samples."""
        # X : (n_samples, *dim, n_channels)
        # Initialization
        arrays = []

        for i in range(len(self.arrays)):
            arrays.append(np.empty(self.in_dims[i]).astype(np.single))

        for i, ID in enumerate(temp_list):
            with np.load(ID) as npzfile:
                for idx in range(len(self.arrays)):
                    x = npzfile[self.arrays[idx]] \
                        .astype(np.single)
                    x = np.expand_dims(x, axis=2)
                    arrays[idx][i, ] = x

        return arrays

# NOTE: Don't change the data generator!

In [ ]:
gen_dir = "/import/software/3ra023/vt22/brats/data/"  # Change if you have copied the data locally on your machine 
array_labels = ["t1", "mask"]  # Available arrays are: 't1', 't1ce', 't2', 'flair', 'mask'.
batch_size = 32

gen_train = DataGenerator(data_path=gen_dir + "training",
                          arrays=array_labels,
                          batch_size=batch_size)

gen_val = DataGenerator(data_path=gen_dir + "validating",
                        arrays=array_labels,
                        batch_size=batch_size)

gen_test = DataGenerator(data_path=gen_dir + "testing",
                         arrays=array_labels,
                         batch_size=batch_size)

# NOTE: What arrays are you using? You can use multiple contrasts as inputs, if you'd like.
# NOTE: Which contrasts do you think are the best to use for tumor segmentation? Try plotting them all.
# NOTE: What batch size are you using? Should you use more? Or less?
# NOTE: Are you using the correct generators for the correct task? Training for training and validating for validating?

### Let's plot some example images from the dataset:

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

fig = plt.figure(figsize=(16.0, 8.0))
fig.subplots_adjust(left=0.001,
                    right=0.9975,
                    top=0.95,
                    bottom=0.005,
                    wspace=0.05,
                    hspace=0.14)

M, N = 2, 4
ax = []
for i in range(M):
    ax.append([None] * N)
    for j in range(N):
        ax[i][j] = plt.subplot2grid((M, N), (i, j), rowspan=1, colspan=1)

imgs = gen_train[0]
idx = [3, 7, 9, 14]  # Plot images in mini-batch with these indices
ii = 0
for j in range(N):
    for i in range(M):
        im = ax[i][j].imshow(imgs[i][idx[ii], :, :, 0], cmap='gray')

        if j == 0:  # Label only on the left
            ax[i][j].set_ylabel(gen_train.arrays[i], fontsize=22)
        if j == N - 1:  # Colorbar only on the right
            divider = make_axes_locatable(ax[i][j])
            cax1 = divider.append_axes("right", size="7%", pad=0.05)
            cbar = plt.colorbar(im, cax=cax1)
    ii += 1    

A quick summary of the data sizes:

In [ ]:
# A quick summary of the data:
print(f"Number of training images : {len(gen_train.file_list)}")
print(f"Training batch size       : {gen_train.in_dims}")

### The dataset preprocessing so far has been to help you, you should not change anything above. However, from now on, take nothing for granted.

In [ ]:
# Import packages important for building and training your model.

import tensorflow.keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Conv2D
from tensorflow.keras.layers import Flatten, Input
from tensorflow.keras.layers import MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.layers import Activation, concatenate
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dropout, UpSampling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras import backend as K
from tensorflow.keras import optimizers

In [ ]:
from tensorflow import Tensor
from tensorflow.keras.layers import Input, Conv2D, ReLU, BatchNormalization, \
                                    Add, AveragePooling2D, Flatten, Dense, UpSampling2D
from tensorflow.keras.models import Model

def build_model():
    filt_size = 2
    input1 = Input(shape=(128, 128, 1))

    conv1 = Conv2D(filt_size, 3, activation='relu', padding='same', kernel_initializer='zeros')(input1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(filt_size * 2, 3, activation='relu', padding='same', kernel_initializer='zeros')(pool1)
    up1 = UpSampling2D(size = (2,2))(conv2)
    conv3 = Conv2D(1, 3, activation='tanh', padding='same', kernel_initializer='zeros')(up1)
    
    return Model(inputs=[input1], outputs=conv3)

# NOTE: Be inspired by the imported layers.
# NOTE: Probably the most famous model architecture for segmentation is the U-Net.

In [ ]:
# Build your model.
model = build_model()
model.summary()

# NOTE: Are the input sizes correct?
# NOTE: Do you have the correct number of input images?
# NOTE: Are the output sizes correct?
# NOTE: Do you have the correct number of output images?
# NOTE: What's the range of the output? Can you use an activation as a regularizer?
# NOTE: Try to imagine the model layer-by-layer and think it through. Is it doing something reasonable?
# NOTE: Are your parameters split evenly inside the model? Try making "too large" layers smaller
# NOTE: Will the model fit into memory? Is the model too small? Is the model too large?

In [ ]:
def dice_coef(y_true, y_pred, smooth=0.001):
    intersection = K.sum(y_true * y_pred, axis=[1, 2, 3])
    union = K.sum(y_true, axis=[1, 2, 3]) + K.sum(y_pred, axis=[1, 2, 3])
    # Note that we want to maximise the Dice coefficient, but if you want to use it as a loss you need the negative
    return -K.mean((2.0 * intersection + smooth) / (union + smooth), axis=0)


learning_rate = 0.0
optim = optimizers.RMSprop(lr=learning_rate)
model.compile(loss="mse",
              optimizer=optim)

# NOTE: Are you satisfied with the optimizer and its parameters?
# NOTE: Try incorporating the DICE score into the model. Is it a loss function? Or a metric?

In [ ]:
n_epochs = 5

for epoch in range(n_epochs):
    training_loss = []
    validating_loss = []
    
    for idx, (t1, mask) in enumerate(gen_train):
        h = model.train_on_batch([t1], mask)
        training_loss.append(h)
    
    for idx, (t1, mask) in enumerate(gen_val):
        validating_loss.append(model.test_on_batch([t1], mask))
        
    print(np.mean(validating_loss))

# NOTE: Plotting the losses helps a lot.
# NOTE: What does plotting the training data tell you? Should you plot something else?
# NOTE: What should one do with the validation data? The fit_generator has a 'validation_data' argument as well.
# NOTE: When should one stop? Did you overtrain? Did you train for long enough?
# NOTE: Think about implementing Early Stopping?

In [ ]:
t1, mask = gen_val[np.random.randint(0, len(gen_val))]
prediction = model.predict([t1])

plt.figure(figsize=(8, 4*batch_size))
for idx in range(batch_size):
    plt.subplot(batch_size, 2, idx * 2 + 1)
    plt.imshow(mask[idx, :, :], cmap='gray')
    plt.title('GT', fontsize=22)
    plt.subplot(batch_size, 2, idx * 2 + 2)
    plt.imshow(prediction[idx, :, :], cmap='gray')
    plt.title('PRED', fontsize=22)
    
# NOTE: What do the predictions mean? What values do they take?
# NOTE: How can your predictions be used as masks?